In [6]:
import requests
from bs4 import BeautifulSoup
import csv
import re

def dms_to_decimal(dms_str):
    matches = re.match(r"(\d+)º (\d+)' (\d+)'' ([NSEW])", dms_str)
    if not matches:
        return None

    degrees, minutes, seconds, direction = matches.groups()
    decimal = float(degrees) + float(minutes)/60 + float(seconds)/3600
    
    if direction in ['S', 'W']:
        decimal = -decimal

    return decimal

url = "https://www.meteoclimatic.net/index/mapinfo/ESCAT?d=20231021&mp=0" 
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

station_links = [link for link in soup.select("a") if link.get('href') and link['href'].startswith('/perfil/')]

station_texts = [link.text.strip() for link in soup.select("a") if link.get('href') and link['href'].startswith('/perfil/')]

station_links

[<a href="/perfil/ESCAT0800000008618A">Refugi Rasos de Peguera (Barcelona)</a>,
 <a href="/perfil/ESCAT0800000008591A">Aiguafreda</a>,
 <a href="/perfil/ESCAT0800000008591B">Aiguafreda Aj.</a>,
 <a href="/perfil/ESCAT0800000008328F">Alella-Mirador</a>,
 <a href="/perfil/ESCAT0800000008350A">Arenys de Mar</a>,
 <a href="/perfil/ESCAT0800000008358A">Arenys de Munt</a>,
 <a href="/perfil/ESCAT0800000008310B">Argentona</a>,
 <a href="/perfil/ESCAT0800000008610A">Avià - Sud</a>,
 <a href="/perfil/ESCAT0800000008915A">Badalona - Bufalà</a>,
 <a href="/perfil/ESCAT0800000008912A">Badalona - Centre</a>,
 <a href="/perfil/ESCAT0800000008911C">Badalona - Dalt la Vila</a>,
 <a href="/perfil/ESCAT0800000008080A">Badalona - Llefià</a>,
 <a href="/perfil/ESCAT0800000008912B">Badalona - Progres</a>,
 <a href="/perfil/ESCAT0800000008695C">Bagà (centre)</a>,
 <a href="/perfil/ESCAT0800000008695A">Bagà (nord)</a>,
 <a href="/perfil/ESCAT0800000008695D">Bagà (sud)</a>,
 <a href="/perfil/ESCAT080000000800

In [7]:
station_texts

['Refugi Rasos de Peguera (Barcelona)',
 'Aiguafreda',
 'Aiguafreda Aj.',
 'Alella-Mirador',
 'Arenys de Mar',
 'Arenys de Munt',
 'Argentona',
 'Avià - Sud',
 'Badalona - Bufalà',
 'Badalona - Centre',
 'Badalona - Dalt la Vila',
 'Badalona - Llefià',
 'Badalona - Progres',
 'Bagà (centre)',
 'Bagà (nord)',
 'Bagà (sud)',
 'Barcelona - Barri Gòtic',
 'Barcelona - betevé',
 'Barcelona - Bon Pastor',
 'Barcelona - Can Bruixa',
 'Barcelona - El Clot',
 'Barcelona - FEDAC Andreu',
 'Barcelona - Fontcoberta',
 'Barcelona - Gràcia',
 'Barcelona - Hosp. Dexeus',
 'Barcelona - Hostafrancs',
 'Barcelona - La Pedrera',
 'Barcelona - La Verneda',
 'Barcelona - Montbau',
 "Barcelona - Pl.de l'Aigua",
 'Barcelona - Plaça Sants',
 'Barcelona - Poblenou',
 'Barcelona - Ps. Sant Joan',
 'Barcelona - Rectoret',
 'Barcelona - Sant Andreu',
 'Barcelona - Sant Gervasi',
 'Barcelona - Sarrià',
 'Barcelona - Tibidabo',
 'Barcelona - Vallcarca',
 'Barcelona Sants-Montjuïc',
 'Bellmunt',
 'Bellmunt - Santuar

In [8]:
results = []

for link in station_links:
    station_name = link.text.strip()  # strip() to remove any unnecessary whitespace
    station_url = "https://www.meteoclimatic.net" + link['href'].strip()  
    response = requests.get(station_url)
    station_soup = BeautifulSoup(response.content, 'html.parser')
    
    lat_lon_elements = station_soup.find_all(string=lambda text: "º" in text and "''" in text)
    
    if lat_lon_elements and len(lat_lon_elements[0].split("\xa0\xa0\xa0\xa0")) >= 2:
        lat_str, lon_str = lat_lon_elements[0].split("\xa0\xa0\xa0\xa0")[0:2]
        latitude_decimal = dms_to_decimal(lat_str)
        longitude_decimal = dms_to_decimal(lon_str)
        
        if latitude_decimal and longitude_decimal:
            results.append([station_name, latitude_decimal, longitude_decimal])

In [9]:
results

[['Refugi Rasos de Peguera (Barcelona)', 42.13611111111111, 1.7625],
 ['Aiguafreda', 41.76888888888889, 2.250277777777778],
 ['Aiguafreda Aj.', 41.770833333333336, 2.2505555555555556],
 ['Alella-Mirador', 41.4925, 2.2913888888888887],
 ['Arenys de Mar', 41.59277777777778, 2.565],
 ['Arenys de Munt', 41.61277777777778, 2.540277777777778],
 ['Argentona', 41.558611111111105, 2.3925],
 ['Avià - Sud', 42.07361111111111, 1.8277777777777777],
 ['Badalona - Bufalà', 41.461111111111116, 2.243611111111111],
 ['Badalona - Centre', 41.44583333333333, 2.2475],
 ['Badalona - Dalt la Vila', 41.45944444444445, 2.2602777777777776],
 ['Badalona - Llefià', 41.43888888888888, 2.225],
 ['Badalona - Progres', 41.44499999999999, 2.243333333333333],
 ['Bagà (centre)', 42.251666666666665, 1.863888888888889],
 ['Bagà (nord)', 42.257777777777775, 1.860277777777778],
 ['Bagà (sud)', 42.24666666666667, 1.8675],
 ['Barcelona - Barri Gòtic', 41.38333333333333, 2.177222222222222],
 ['Barcelona - betevé', 41.400555555

In [10]:
len(results)

443

In [11]:
with open('stations_coordinates.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Observatori', 'Latitud', 'Longitud'])
    csvwriter.writerows(results)